# Imports

In [6]:
import pysweep
import pysweep.databackends.debug
import pysweep.databackends.list_backend
import pysweep.databackends.qcodes
import pysweep.core.measurementfunctions
from pysweep.core.sweepobject import SweepObject
from pysweep.core.measurementfunctions import MakeMeasurementFunction
from pysweep.databackends.base import DataParameterFixedAxis
from pysweep.databackends.base import DataParameter
import pysweep.convenience as conv

import qcodes as qc
from qcodes.dataset.measurements import Measurement
import qcodes.dataset.plotting

import cqed.custom_pysweep_functions.vna as cvna
import cqed.custom_pysweep_functions.magnet as cmgnt

import numpy as np
import matplotlib.pyplot as plt

In [4]:
def create_local_dbase_in(folder_name='general', db_name='experiments.db', data_dir='D:/Data'):
    """    
    Initialise or create a QCoDeS database in D:/Data/folder_name/db_name 
    If the directory does not exist, it is created.
    Set the QCoDeS database location to point to the specified database.
    
    Inputs:
    folder_name (str): Name of the subfolder in D:/Data where db is stored. 
                       Can also be a path to a subfolder, e.g. general/sample1 leads to db in D:/Data/general/sample1 
    db_name (str):     Name of database, including .db
    
    data_dt (str):     Parent folder for all database files. 
    """
    from pathlib import Path
    
    dest_dir = Path(data_dir, folder_name)
    if not dest_dir.exists():
        dest_dir.mkdir(parents=True)        
    dest_path = Path(dest_dir, db_name)
    qc.initialise_or_create_database_at(dest_path)
    qc.config['core']['db_location'] = dest_path

# Set up station and QCoDeS database

## Set up measurement station

for now, we keep the initalisation file <br>
**TODO:** move station import to YAML file  <br>
https://qcodes.github.io/Qcodes/examples/Station.html#Using-Station-with-a-YAML-configuration-file

### K1

In [ ]:
%run -i "D:\OneDrive\Setups\LK1\LK1_code\Code\LK1\LK1\init_station_VNA.py"

### K2

In [ ]:
%run -i "D:\OneDrive\Setups\LK2\LK2_code\Code\LK2\LK2\init_station_VNA.py"

### Small addition to VNA, to be able to set electrical delay via software

These functions will be implemented in the ZNB20 driver in one of the future qcodes releases. Remove these redundant lines then.

In [ ]:
# This parameter allows to get and set the electrical delay of port 2 of the VNA. 
# Used to get rid of the phase roll due to the cabel length.
station.vna.S21.add_parameter(name='electrical_delay', 
                              get_cmd='SENS1:CORR:EDEL2:TIME?',
                              set_cmd='SENS1:CORR:EDEL2:TIME {}',
                              get_parser=float,
                              unit = 's')

In [ ]:
# This parameter allows to automatically set the electrical delay to get rid of the phase roll due to the length of the cable. 
# By default, the delay is added to the active (i.e. emitting) port of the VNA.
station.vna.S21.add_function('set_electrical_delay_auto',
                            call_cmd='SENS1:CORR:EDEL:AUTO ONCE')

## Pass measurement station to pysweep

In [ ]:
pysweep.STATION = station

## Name and initialize database

In [ ]:
# To place the database in D:/Data/MyExperiment/ use:
folder = 'MyExperiment'

# To place the database in a more nested folder structure, e.g. D:/Data/MyExperiment/TestSample/FirstMeasurements/, use:
# folder = 'MyExperiment/TestSample/FirstMeasurements'

# Name your database
database_name = 'yyyy_mm_dd_Run1'

create_local_dbase_in(folder_name=folder, db_name='%s.db' %(database_name))

qc.config['core']['db_location']

# Define pysweep functions

In [ ]:
# Function called before beginning of measurement loop.
# This is a good place to define the VNA sweep parameters. Can also be made a function with changable parameters.

def init_measurement(d):
    # This block only to always set the electrical delay before each measurement.
    # Large frequency range gives a more accurate automatic setting.
    station.vna.S21.start(4e9)
    station.vna.S21.stop(8e9)
    station.vna.S21.set_electrical_delay_auto()
    # End of autodelay set
    
    station.vna.S21.start(4e9)
    station.vna.S21.stop(5e9)
    station.vna.S21.npts(1001)
    
    station.vna.S21.power(-20)
    station.vna.S21.bandwidth(1e3)
    station.vna.S21.avg(10)
    
    print('Starting measurement.')
    
# Function called after the end of the measurement.
def end_measurement(d):
    print('Measurement finished.')

# Single VNA trace

This just records a single VNA trace with the VNA parameters currently set or set (if specified) in the init_measurement function.

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='single_VNA_trace', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cvna.return_vna_trace,
                 databackend = pysweep.databackends.qcodes.DataBackend(meas))

For plotting of the generated dataset in the notebook directly

In [ ]:
qc.dataset.plotting.plot_dataset(result.datasaver.dataset)

# Power sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_pwr', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cvna.return_vna_trace, 
                  pysweep.sweep_object(station.vna.S21.power, np.arange(-50, -15, 5)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

# Gate sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_gate', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cvna.return_vna_trace, 
                  pysweep.sweep_object(station.ivvi.dac1, np.arange(0, 1010, 10)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

# Magnetic field sweep

## Setting of x, y, z magnetic fields

In [ ]:
station.mgnt.field_ramp_rate(FieldVector(x=3e-3, y=3e-3, z=3e-3))

In [ ]:
# Get current field strength of x, y, and z component.
print(station.mgnt.x_measured())
print(station.mgnt.y_measured())
print(station.mgnt.z_measured())

In [ ]:
# Set all target field components to some (reasonable) value and ramp magnetic field.
# After ramping, check field strength
station.mgnt.x_target(0.0e-3)
station.mgnt.y_target(0.0e-3)
station.mgnt.z_target(0.1e-3)

station.mgnt.ramp(mode='safe')

print(station.mgnt.r_measured())
print(station.mgnt.phi_measured())
print(station.mgnt.theta_measured())

## The following functions are under testing, ask Lukas G./S.

The sweep functions below explicitly calculate x, y, and z components from the spherical component input. This is done for greater control of the actual sweeping direction of the magnet

For the actual measurement, we make use of the '+' functionality of pysweep, to record the measured magnetic field components along with the desired setpoints.

### Phi sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt-phi', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_components + cvna.return_vna_trace, 
                  cmgnt.sweep_phi(r = 3.0e-3, theta = 90, points = np.arange(0, 360, 5)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

## Theta sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt-theta', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_orientation + cvna.return_vna_trace, 
                  cmgnt.sweep_theta(r = 3.0e-3, phi = 90, points = np.arange(0, 180, 5)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

## r sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt-r', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_orientation + cvna.return_vna_trace, 
                  cmgnt.sweep_r(phi = 90, theta = 0, points = np.arange(0, 20e-3, 2e-3)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))